In [1]:
from bs4 import BeautifulSoup
import urllib
import urllib2
import re
import time
import numpy as np
import networkx as nx
import community # pip install python-louvain
import csv
import requests
import xml.etree.ElementTree as ET
import json
import collections
import cPickle as pickle
from IPython.display import clear_output

class Anime:
    def __init__(self, id_num):
        self.id_num = id_num

In [2]:
# Initial settings
threshold = 2;
queue = []
edge_list = []
anime_list = {}

In [3]:
# Get recommendation data, takes ~2-3 hours

queue.append(2167) # Add Clannad <3
# queue.insert(0, anime_id) # Run if connection cuts out

while len(queue) > 0:
    anime_id = queue.pop(0)
    anime = Anime(anime_id)
    
    # Retrieve website data
    url = "https://myanimelist.net/anime/" + str(anime_id)
    rec_url = "https://myanimelist.net/anime/" + str(anime_id) +"/_/userrecs"
    try:
        mal_file = urllib2.urlopen(url)
        mal_html = mal_file.read()
        mal_file.close()
        
        rec_mal_file = urllib2.urlopen(rec_url)
        rec_mal_html = rec_mal_file.read()
        rec_mal_file.close()  
    except:
        time.sleep(60)
        print "Retrying..."
        
        mal_file = urllib2.urlopen(url)
        mal_html = mal_file.read()
        mal_file.close()
        
        rec_mal_file = urllib2.urlopen(rec_url)
        rec_mal_html = rec_mal_file.read()
        rec_mal_file.close() 
        
    soup = BeautifulSoup(mal_html, 'lxml')
    rec_soup = BeautifulSoup(rec_mal_html, 'lxml')
    # print soup.prettify()

    # Get anime name
    name = soup.find('span', itemprop='name').get_text()
    anime.name = name

    # Get synopsis
    synopsis = soup.find('span', itemprop='description').get_text()
    anime.synopsis = synopsis

    # Get score
    score_text = soup.find('div', "score").get_text()
    score = float(score_text)
    anime.score = score

    # Get rank
    rank_text = soup.find('span', "numbers ranked").get_text()
    rank_num = re.findall('\d+', rank_text)
    if len(rank_num) > 0:
        rank = int(rank_num[0])
    else:
        rank = 0
    anime.rank = rank

    # Get popularity
    pop_text = soup.find('span', "numbers popularity").get_text()
    pop_num = re.findall('\d+', pop_text)
    if len(pop_num) > 0:
        popularity = int(pop_num[0])
    else:
        popularity = 0
    anime.popularity = popularity

    # Get members
    mem_text = soup.find('span', "numbers members").get_text()
    mem_text = mem_text.replace(',', '')
    mem_num = re.findall('\d+', mem_text)
    if len(mem_num) > 0:
        members = int(mem_num[0])
    else:
        members = 0
    anime.members = members

    # Get episodes
    stats = soup.find_all('div', "spaceit")
    for s in stats:
        stat = s.get_text()
        if "Episodes:" in stat:
            eps_num = re.findall('\d+', stat)
            if len(eps_num) > 0:
                episodes = int(eps_num[0])
            else:
                episodes = 0
    anime.episodes = episodes

    # Get season
    season_name = soup.select('a[href^="https://myanimelist.net/anime/season/"]')
    if len(season_name) > 0:
        season = season_name[0].get_text()
    else:
        season = ''
    anime.season = season

    # Get genres
    genre_list = [];
    genre_names = [];
    genres = soup.select('a[href^="/anime/genre/"]')
    for g in genres:
        genre_text = g.get('href')
        genre = re.findall('\d+', genre_text)
        genre_list.append(int(genre[0]))
        genre_names.append(g.get_text())
    anime.genre_names = genre_names
    anime.genre_list = genre_list

    # Download image
    img_url= soup.find('img', itemprop='image').get('src')
    # urllib.urlretrieve(str(img_url), 'images/' + str(anime_id) + '.jpg')
    anime.img_url = img_url

    # Get recommendation numbers
    rec_tags = rec_soup.find_all('a', "js-similar-recommendations-button")
    rec_list = [];
    for rec_tag_strs in rec_tags:
        rec_text = rec_tag_strs.get_text()
        parsed_rec = re.findall('\d+', rec_text)
        num_recs = int(parsed_rec[0])
        if num_recs >= threshold:
            rec_list.append(num_recs)
    anime.rec_list = rec_list

    # Get recommendation ids
    recs = rec_soup.find_all('a', "hoverinfo_trigger")
    id_list = []
    for rec_names in recs:
        rec_name = rec_names.get('href')
        rec_anime_id = re.findall('\d+', rec_name)
        id_list.append(int(rec_anime_id[0]))
    id_list = id_list[:len(rec_list)]
    anime.id_list = id_list
    
    # Add anime to visited dictionary, new anime to queue, new edges to edge list
    anime_list[anime_id] = anime
    for index, id_num in enumerate(id_list):
        if not anime_list.has_key(id_num):
            if id_num not in queue:
                queue.append(id_num)
            edge_list.append([anime_id, id_num, rec_list[index]])
    
    clear_output()
    print anime.name + ' | ' + str(len(queue))
    time.sleep(2)

Tales of Phantasia The Animation | 0


In [4]:
# Post-processing and data storage

# Generate communities
G = nx.Graph()
for k in anime_list:
    G.add_node(k)
for e in edge_list:
    G.add_edge(e[0],e[1],weight=e[2])
partition = community.best_partition(G)

# Add communities to anime data
count = 0
for com in set(partition.values()) :
    count = count + 1
    list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com]
    for n in list_nodes:
        a = anime_list[n]
        a.community = count

# Generate .csv files for import into Gephi
# Export edges
with open('edges.csv', 'wb') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(["Source", "Target", "Weight", "Type"])
    for e in edge_list:
        line = list(e)
        line.append("Undirected")
        writer.writerow(line)

# Export nodes
with open('nodes.csv', 'wb') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(["ID", "Label", "Image", "Community"])
    for cur_id, a in anime_list.items():
        writer.writerow([cur_id, a.name.encode('utf-8'), a.img_url, a.community])

# Save anime list data
pickle.dump(anime_list, open('save.p', 'wb'))
  
# Draw network
# pos = nx.spring_layout(G)
# size = float(len(set(partition.values())))
# count = 0
# for com in set(partition.values()) :
#     count = count + 1
#     list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com]
#     for n in list_nodes:
#         nx.draw_networkx_nodes(G, pos, list_nodes, node_size = 20, node_color = str(count / size))
# nx.draw_networkx_edges(G,pos, alpha=0.5)
# plt.show()
        
# Export all node data       
# with open('nodes.csv', 'wb') as csv_file:
#     writer = csv.writer(csv_file, delimiter=',')
#     writer.writerow(["ID", "Label", "Score", "Rank", "Popularity", "Members", 
#                      "Episodes", "Season", "Genres", "Genre IDS", "MAL Link", 
#                      "Image", "Synopsis", "Community"])
#     for cur_id, a in anime_list.items():
#         genre_str = '';
#         genre_id_str = '';
#         for g in a.genre_names:
#             if genre_str is not '':
#                 genre_str = genre_str + ', '
#             genre_str = genre_str + g
#         for g_id in a.genre_list:
#             if genre_id_str is not '':
#                 genre_id_str = genre_id_str + ', '
#             genre_id_str = genre_id_str + str(g_id)
#         cur_url = '<a href="https://myanimelist.net/anime/' + str(cur_id) + '">'+ a.name.encode('utf-8') + '</a>'
#         cur_img_url = '<img src="' + a.img_url + '" alt="' + a.name.encode('utf-8') + '">'
#         writer.writerow([cur_id, a.name.encode('utf-8'), a.score, "#"+str(a.rank), "#"+str(a.popularity), 
#                          '{:,}'.format(a.members), a.episodes, a.season, genre_str, genre_id_str, cur_url, 
#                          a.img_url, a.synopsis.encode('utf-8'), a.community])


In [3]:
# Instructions for Gephi (for consistency)
# 1. Import nodes, import edges
# 2. Run OpenOrd layout, then run ForceAtlas2 until stable
# 3. Rotate until at a desirable position
# 4. Change node color to be based on community, use the Fluorescent palette
# 5. Change node size to be based on degree, with min = 1, max = 5, and a circular distribution
# 6. Change node text size to be based on degree, with min = 1, max = 5, and a circular distribution
# 7. Slide text size to the smallest font, set font size to 14, then run LabelAdjust
# 8. Modify any of the above parameters for a more preferred layout
# 9. Export to sigma.js template, uncheck the replace node ids with numbers box


In [7]:
# Export generic network graph showing detected communities

# Import anime_list object
anime_list = pickle.load(open('save.p', 'rb'))

# Load exported data file from Gephi
with open('network/data.json') as data_file:    
    sigma_data = json.load(data_file)
    data_file.close()

# Modify attributes
for d in sigma_data['nodes']:
    cur_id = int(d['id'])
    a = anime_list.get(cur_id)
    attr = collections.OrderedDict()
    attr['Score'] = a.score
    attr['Rank'] = '#' + str(a.rank)
    attr['Popularity'] = '#' + str(a.popularity)
    attr['Members'] = '{:,}'.format(a.members)
    attr['Episodes'] = a.episodes
    attr['Season'] = a.season
    genre_str = '';
    for g in a.genre_names:
        if genre_str is not '':
            genre_str = genre_str + ', '
        genre_str = genre_str + g
    attr['Genres'] = genre_str
    cur_url = '<a href="https://myanimelist.net/anime/' + str(cur_id) + '">' + a.name.encode('utf-8') + '</a>'
    attr['MAL Link'] = cur_url
    attr['Synopsis'] = a.synopsis.encode('utf-8')
    attr['Image'] = a.img_url
    d['attributes'] = attr
    
for d in sigma_data['edges']:
    d['color'] = 'rgb(150,150,150)'

# Export new data file
with open('MALmap/data.json', 'w') as outfile:  
    json.dump(sigma_data, outfile)

In [4]:
# Export user's MAL network graph

# Import anime_list object
anime_list = pickle.load(open('save.p', 'rb'))

# username = "igfod13"
username = raw_input("Username: ")

# Load user's anime list
url = "https://myanimelist.net/malappinfo.php?status=all&type=anime&u=" + username
response = requests.get(url)
# print (response.status_code)
tree = ET.fromstring(response.content)

# Retrieve user data
user_list = {}
user_score_list ={}
for anime in tree:
    for item in anime:
        if item.tag == 'series_animedb_id':
            temp_id = int(item.text)
        if item.tag == 'my_status':
            temp_status = int(item.text)
        if item.tag == 'my_score':
            temp_score = int(item.text)
    if anime.tag == 'anime':
        user_list[temp_id] = temp_status
        user_score_list[temp_id] = temp_score

# Get user's score distribution
user_data = []    
for i, a in user_score_list.items():
    if a != 0:
        user_data.append(a)
a = np.array(user_data)
user_percentile = np.percentile(a, 65)

# Get total score distribution
data = []
for cur_id, a in anime_list.items():
    data.append(a.score)
a = np.array(data)
p1 = np.percentile(a, 90)
p2 = np.percentile(a, 70)
p3 = np.percentile(a, 40) 

# MAL status IDS: 1 = watching, 2 = completed, 3 = on hold, 4 = dropped, 6 = plan to watch
# Additional IDS: 9 = highly ranked by user, 10-13 = top 0-10/10-30/30-60/60-100% ranked unwatched
for cur_id, a in anime_list.items():
    if user_list.has_key(cur_id):
        if user_list[cur_id] == 2 and user_score_list[cur_id] > user_percentile:
                a.user_status = 9
        else:
            a.user_status = user_list[cur_id]
    else:
        if a.score > p1:
            a.user_status = 10
        elif a.score > p2:
            a.user_status = 11
        elif a.score > p3:
            a.user_status = 12
        else:
            a.user_status = 13
            
# Load exported data file from Gephi
# with open('network/data.json') as data_file:
with open('data.json') as data_file:  
    sigma_data = json.load(data_file)
    data_file.close()

# Modify attributes
for d in sigma_data['nodes']:
    cur_id = int(d['id'])
    a = anime_list.get(cur_id)
    attr = collections.OrderedDict()
    attr['Score'] = a.score
    attr['Rank'] = '#' + str(a.rank)
    attr['Popularity'] = '#' + str(a.popularity)
    attr['Members'] = '{:,}'.format(a.members)
    attr['Episodes'] = a.episodes
    attr['Season'] = a.season
    genre_str = '';
    for g in a.genre_names:
        if genre_str is not '':
            genre_str = genre_str + ', '
        genre_str = genre_str + g
    attr['Genres'] = genre_str
    cur_url = '<a href="https://myanimelist.net/anime/' + str(cur_id) + '">' + a.name.encode('utf-8') + '</a>'
    attr['MAL Link'] = cur_url
    attr['Synopsis'] = a.synopsis.encode('utf-8')
    attr['Image'] = a.img_url
    d['attributes'] = attr
    # Set node colors
    if a.user_status == 1:
        d['color'] = 'rgb(45,175,55)'
    elif a.user_status == 2:
        d['color'] = 'rgb(40,80,210)'
    elif a.user_status == 3:
        d['color'] = 'rgb(250,210,85)'
    elif a.user_status == 4:
        d['color'] = 'rgb(160,45,50)'
    elif a.user_status == 6:
        d['color'] = 'rgb(255,155,45)'
    elif a.user_status == 9:
        d['color'] = 'rgb(50,190,255)'
    elif a.user_status == 10:
        d['color'] = 'rgb(195,195,195)'
    elif a.user_status == 11:
        d['color'] = 'rgb(155,155,155)'
    elif a.user_status == 12:
        d['color'] = 'rgb(115,115,115)'
    else:
        d['color'] = 'rgb(75,75,75)'
    
for d in sigma_data['edges']:
    d['color'] = 'rgb(75,75,75)'

# Export new data file
# with open('MALmap/data.json', 'w') as outfile:  
with open('data.json', 'w') as outfile:  
    json.dump(sigma_data, outfile)

Username: igfod13
